In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
import os
import numpy as np
import librosa
import tensorflow as tf
import pickle
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve
from scipy.fftpack import dct
import gc

In [ ]:
X = np.load('../SavedFeatures/X_mfcc.npy')
y = np.load('../SavedFeatures/y_mfcc.npy')
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Đổi trường dữ liệu 
X_train = X_train.reshape(X_train.shape[0], -1)  # to 2D
X_val = X_val.reshape(X_val.shape[0], -1)  # to 2D

del X, y
gc.collect()

In [7]:
# Base models
base_models = [
    ('svm', make_pipeline(StandardScaler(), SVC(kernel='rbf', probability=True))),
    ('xgb', XGBClassifier()),
    ('rf', RandomForestClassifier(n_estimators=150, random_state=42))
]

In [ ]:
meta_model = DecisionTreeClassifier(max_depth = 20)

In [ ]:
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)
stacking_model.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('svm',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('svc',
                                                 SVC(probability=True))])),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_t...
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...)),
                               ('rf',
                                RandomForestClassifier(n_estimators=150,
                                                       random_state=42))],
                   final_estimator=DecisionTreeClassifier(max_depth=20))

In [ ]:
y_val_predictions = stacking_model.predict(X_val) 
y_val_probabilities = stacking_model.predict_proba(X_val) 
accuracy = accuracy_score(y_val, y_val_predictions)

In [ ]:
f1_class_0 = f1_score(y_val, y_val_predictions, pos_label=0)
f1_class_1 = f1_score(y_val, y_val_predictions, pos_label=1)
f1_score_average = (f1_class_0 + f1_class_1) / 2

# Tính EER 

y_val_probabilities1 = y_val_probabilities[:, 1]  
fpr1, tpr1, thresholds1 = roc_curve(y_val, y_val_probabilities1)
frr1 = 1 - tpr1
eer_index1 = np.nanargmin(np.abs(fpr1 - frr1))
eer1 = fpr1[eer_index1]
eer_threshold1 = thresholds1[eer_index1]


y_val_probabilities0 = y_val_probabilities[:, 0] 
fpr0, tpr0, thresholds0 = roc_curve(y_val, y_val_probabilities0)
frr0 = 1 - tpr0
eer_index0 = np.nanargmin(np.abs(fpr0 - frr0))
eer0 = fpr0[eer_index0]
eer_threshold0 = thresholds0[eer_index0]




print("====================Stacking Model Evaluation====================")
print(f"Accuracy: {accuracy:.5f}")
print(f"F1-Score (Average): {f1_score_average:.5f}")
print("\nClassification Report:\n", classification_report(y_val, y_val_predictions))
print("EER of Class 0 (Bonafide): ---------")
print(f"Equal Error Rate (EER): {eer0:.5f}")
print(f"EER Threshold: {eer_threshold0:.5f}")
print("EER of Class 1 (Spoof): ---------")
print(f"Equal Error Rate (EER): {eer1:.5f}")
print(f"EER Threshold: {eer_threshold1:.5f}")

In [ ]:
with open('../SavedModels/MFCC_stacking_model.pkl', 'wb') as f:
    pickle.dump(stacking_model, f)


Model saved successfully!


In [ ]:
del stacking_model, X_train, X_val, y_train, y_val
gc.collect()